# Validação: Sentence Transformers

**SBERT - Embeddings para Classificação**

## 📊 Abordagens
1. Embeddings + LogisticRegression
2. Embeddings + SVM
3. Embeddings + LightGBM
4. SetFit (few-shot fine-tuning)

## 🎯 Modelos Testados
- `paraphrase-multilingual-MiniLM-L12-v2` (multilingual)
- `all-MiniLM-L6-v2` (rápido, inglês)
- Custom embedding com fine-tuning

---

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)

if os.path.exists('/kaggle/input'):
    DATA_DIR = '/kaggle/input/competitions/spr-2026-mammography-report-classification'
    def find_model_path():
        base = '/kaggle/input'
        def search_dir(directory, depth=0, max_depth=10):
            if depth > max_depth: return None
            try:
                for item in os.listdir(directory):
                    path = os.path.join(directory, item)
                    if os.path.isdir(path) and os.path.exists(os.path.join(path, 'modules.json')):
                        return path
                    result = search_dir(path, depth + 1, max_depth) if os.path.isdir(path) else None
                    if result: return result
            except: pass
            return None
        return search_dir(base)
    MODEL_PATH = find_model_path()
else:
    DATA_DIR = '../data'
    MODEL_PATH = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

print(f'Model: {MODEL_PATH}')

In [ ]:
# ===== DADOS =====
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].tolist(),
    train_df['target'].tolist(),
    test_size=0.2,
    stratify=train_df['target'],
    random_state=SEED
)

print(f'Train: {len(train_texts)}, Val: {len(val_texts)}')

In [ ]:
# ===== CARREGAR SENTENCE TRANSFORMER =====
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(MODEL_PATH)
print(f'Model loaded: {model.get_sentence_embedding_dimension()} dims')

In [ ]:
# ===== GERAR EMBEDDINGS =====
print('Gerando embeddings...')

train_embeddings = model.encode(train_texts, show_progress_bar=True)
val_embeddings = model.encode(val_texts, show_progress_bar=True)

print(f'Train embeddings: {train_embeddings.shape}')
print(f'Val embeddings: {val_embeddings.shape}')

In [ ]:
# ===== TESTE 1: LOGISTIC REGRESSION =====
print("\n" + "="*50)
print("Logistic Regression")
print("="*50)

lr_model = LogisticRegression(max_iter=1000, random_state=SEED, class_weight='balanced')
lr_model.fit(train_embeddings, train_labels)

lr_preds = lr_model.predict(val_embeddings)
lr_f1 = f1_score(val_labels, lr_preds, average='macro')

print(f'F1-Macro: {lr_f1:.5f}')
print(classification_report(val_labels, lr_preds))

In [ ]:
# ===== TESTE 2: SVM =====
print("\n" + "="*50)
print("SVM (RBF kernel)")
print("="*50)

svm_model = SVC(kernel='rbf', random_state=SEED, class_weight='balanced')
svm_model.fit(train_embeddings, train_labels)

svm_preds = svm_model.predict(val_embeddings)
svm_f1 = f1_score(val_labels, svm_preds, average='macro')

print(f'F1-Macro: {svm_f1:.5f}')
print(classification_report(val_labels, svm_preds))

In [ ]:
# ===== TESTE 3: LIGHTGBM =====
print("\n" + "="*50)
print("LightGBM")
print("="*50)

try:
    import lightgbm as lgb
    
    lgb_model = lgb.LGBMClassifier(
        n_estimators=200,
        learning_rate=0.1,
        num_leaves=31,
        random_state=SEED,
        class_weight='balanced',
        verbose=-1
    )
    lgb_model.fit(train_embeddings, train_labels)
    
    lgb_preds = lgb_model.predict(val_embeddings)
    lgb_f1 = f1_score(val_labels, lgb_preds, average='macro')
    
    print(f'F1-Macro: {lgb_f1:.5f}')
    print(classification_report(val_labels, lgb_preds))
except ImportError:
    print('LightGBM não instalado')
    lgb_f1 = 0

In [ ]:
# ===== TESTE 4: SETFIT (Few-shot) =====
print("\n" + "="*50)
print("SetFit (Few-shot fine-tuning)")
print("="*50)

try:
    from setfit import SetFitModel, SetFitTrainer
    from datasets import Dataset
    
    # Criar dataset
    train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
    val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})
    
    # Modelo SetFit
    setfit_model = SetFitModel.from_pretrained(MODEL_PATH)
    
    # Trainer
    trainer = SetFitTrainer(
        model=setfit_model,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        num_iterations=20,
        num_epochs=1,
        batch_size=16,
    )
    
    trainer.train()
    
    setfit_preds = setfit_model.predict(val_texts)
    setfit_f1 = f1_score(val_labels, setfit_preds, average='macro')
    
    print(f'F1-Macro: {setfit_f1:.5f}')
except ImportError:
    print('SetFit não instalado')
    setfit_f1 = 0

In [ ]:
# ===== RESUMO =====
print("\n" + "="*60)
print("📊 RESUMO - Sentence Transformers Validation")
print("="*60)

results = [
    ('LogisticRegression', lr_f1),
    ('SVM (RBF)', svm_f1),
    ('LightGBM', lgb_f1),
    ('SetFit', setfit_f1),
]

for name, f1 in sorted(results, key=lambda x: -x[1]):
    if f1 > 0:
        print(f"{name:<20} {f1:.5f}")

print(f"\n📝 Referência (TF-IDF): 0.77885")
print(f"📝 Referência (BERTimbau v4): 0.82073")

In [ ]:
# ===== INSIGHTS =====
print("""
📝 INSIGHTS - Sentence Transformers
====================================

1. **Embeddings pré-treinados:**
   - Rápidos para gerar
   - Consistentes entre runs

2. **Comparação de classificadores:**
   - [PREENCHER]

3. **SetFit:**
   - Fine-tuning sem grandes GPUs
   - Bom para few-shot

4. **Uso em Ensemble:**
   - Pode complementar transformers full
   - Rápido para voting ensemble
""")